# utils

In [2]:
%matplotlib tk

import numpy as np
import holopy as hp
from holopy.scattering import calc_holo, Sphere

def l2_loss(original, reconstructed):
    # Check that the images have the same shape
    assert original.shape == reconstructed.shape, "The original and reconstructed images must have the same shape"

    # Calculate the L2 loss
    loss = 0
    for i in range(original.shape[0]):
        for j in range(original.shape[1]):
            loss += (original[i,j] - reconstructed[i,j]) ** 2
    loss = np.sqrt(loss)
    n_pixels = original.shape[0] * original.shape[1]
    return loss, n_pixels

def l2_loss_zero(original, reconstructed):
    # Check that the images have the same shape
    assert original.shape == reconstructed.shape, "The original and reconstructed images must have the same shape"

    # Calculate the L2 loss
    loss = 0
    for i in range(original.shape[0]):
        for j in range(original.shape[1]):
            if original[i,j] == 0:
                loss += (original[i,j] - reconstructed[i,j]) ** 2
    loss = np.sqrt(loss)
    n_pixels = np.sum(original == 0)
    return loss, n_pixels

def ssim_loss(img1_gray, img2_gray, L=255):

    # Compute the local mean of the images
    mu1 = np.mean(img1_gray)
    mu2 = np.mean(img2_gray)

    # Compute the local variance of the images
    sigma1_sq = np.mean((img1_gray - mu1)**2)
    sigma2_sq = np.mean((img2_gray - mu2)**2)
    sigma12 = np.mean((img1_gray - mu1)*(img2_gray - mu2))

    # Compute the luminance component of the SSIM
    c1 = (0.01 * L)**2
    lum = (2 * mu1 * mu2 + c1) / (mu1**2 + mu2**2 + c1)

    # Compute the contrast component of the SSIM
    c2 = (0.03 * L)**2
    con = (2 * sigma1_sq * sigma2_sq + c2) / (sigma1_sq**2 + sigma2_sq**2 + c2)

    # Compute the structure component of the SSIM
    c3 = c2 / 2
    struc = (sigma12 + c3) / (sigma1_sq * sigma2_sq + c3)

    # Compute the final SSIM value
    ssim = lum * con * struc

    return ssim


# draw a circle of radius 5 in the middle of the image
def draw_circle(x, y, r, original):
    for i in range(x-r, x+r):
        for j in range(y-r, y+r):
            if (i-x)**2 + (j-y)**2 < r**2:
                original[i,j] = 1
    return original

# Drawing a lineplot of Losses

In [48]:
DETECTOR_SHAPE = [52, 62]
SPACING = 0.5
RADIUS = 10
MEDIUM_INDEX = 1
WAVELENGTH = 15


params = {
    'RADIUS': RADIUS,
    'MEDIUM_INDEX': MEDIUM_INDEX,
    'WAVELENGTH': WAVELENGTH,
    'DETECTOR_SHAPE': DETECTOR_SHAPE,
    'SPACING': SPACING,
    
}

def calculate_loss(DETECTOR_SHAPE, SPACING, RADIUS, MEDIUM_INDEX, WAVELENGTH):

    CENTER = ((DETECTOR_SHAPE[0]*SPACING)/2, (DETECTOR_SHAPE[1]*SPACING)/2, 15)
    RECONSTRUCTION_PLANE = CENTER[2]/SPACING
    CIRCLE_RADIUS = RADIUS
    CIRCLE_CENTER_X = int(DETECTOR_SHAPE[0]/2)
    CIRCLE_CENTER_Y = int(DETECTOR_SHAPE[1]/2)


    sphere = Sphere(
        n=1.59, # refraction of the sphere
        r=RADIUS, 
        center=CENTER 
    )

    medium_index = MEDIUM_INDEX
    illum_wavelen = WAVELENGTH
    illum_polarization = (1, 0) # linear polarization along x axis
    detector = hp.detector_grid(shape=DETECTOR_SHAPE, 
                                spacing=SPACING) 

    holo = calc_holo(
        detector, 
        sphere, 
        medium_index, 
        illum_wavelen, 
        illum_polarization, 
        theory='auto' # use the default theory of sphere
    )
    # hp.show(holo)

    zstack = np.linspace(0, 60, 121)
    rec_vol = hp.propagate(holo, zstack)
    # hp.show(rec_vol)

    # recontruct the image
    recontructed = hp.propagate(holo, RECONSTRUCTION_PLANE)
    recontructed = np.abs(recontructed)
    recontructed = recontructed.to_numpy()

    # draw a circle of radius 5 in the middle of the image
    SIZE = recontructed.shape
    original = np.zeros(SIZE)
    original = draw_circle(CIRCLE_CENTER_X, CIRCLE_CENTER_Y, CIRCLE_RADIUS, original)

    # normalize the two images to be between 0 and 1
    original = original / np.max(original)
    recontructed = recontructed / np.max(recontructed)
    # hp.show(recontructed)
    # hp.show(original)
    

    loss, pixel_number = l2_loss(original, recontructed)
    loss_z, pixel_number_z = l2_loss_zero(original, recontructed)

    fake = np.zeros(SIZE)
    loss_fake, pixel_number_fake = l2_loss(fake, recontructed)
    
    # inverse is the opposite of the original image
    inverse = np.ones(SIZE) - original
    loss_inverse, pixel_number_inverse = l2_loss(inverse, original)

    # uses the SSIM function to calculate the SSIM loss
    loss_ssim = ssim_loss(original, recontructed)

    return loss/pixel_number, loss_z/pixel_number_z, loss_fake/pixel_number_fake, loss_ssim, loss_inverse/pixel_number_inverse

# save losses results
losses = []
losses_z = []
losses_fake = []
losses_ssim = []
losses_inverse = []
# instantiate the SIZE
SIZE = [52, 62]
# cicle over possible values for SIZE
# use tqdm to show the progress bar
from tqdm import tqdm
for i in tqdm(range(1, 100)):
    SIZE[0] = SIZE[0] + 1
    SIZE[1] = SIZE[1] + 1
    params['DETECTOR_SHAPE'] = SIZE
    results = calculate_loss(**params)
    losses.append(results[0])
    losses_z.append(results[1])
    losses_fake.append(results[2])
    losses_ssim.append(results[3])
    losses_inverse.append(results[4])

# scale the results multiplying by 1000
losses = [i*1000 for i in losses]
losses_z = [i*1000 for i in losses_z]

# inlude the matplotlib library
import matplotlib.pyplot as plt

# draw the results in line plot with two colors
fig, ax = plt.subplots()
ax.plot(losses, color='red')
ax.plot(losses_z, color='blue')
# ax.plot(losses_fake, color='green')
# ax.plot(losses_ssim, color='green')
# ax.plot(losses_inverse, color='black')
ax.set_xlabel('SIZE')
ax.set_ylabel('loss')
ax.set_title('losses')
plt.show()


100%|██████████| 99/99 [01:08<00:00,  1.44it/s]


In [53]:
# draw the results in line plot with two colors
fig, ax = plt.subplots()
ax.plot(losses, color='red')
ax.plot(losses_z, color='blue')
# ax.plot(losses_fake, color='black')
# add a legend formula of the losses described in the legend
# - "loss" have formula: $loss = \frac{1}{n} \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}$
# - "loss_z" have formula: $loss_z = \frac{1}{||\big{1}||}\sqrt{\sum_{i=1}^{n} (x_i - y_i)^2 * \big{1}}$
#       where ||\big{1}|| is the number of pixels with value 0 in the original image
#       and \big{1} is a vector of 0 and 1 with 1 in the pixels with value 0 in the original image
#       and 0 in the pixels with value 1 in the original image
ax.legend(['$L2^{*}_{loss}$', '$L2^{*}_{loss\_zero}$', '$L2^{*}_{loss\_fake}$'])
# add labels and title
ax.set_xlabel('SIZE')
ax.set_ylabel('loss')
ax.set_title('losses')
# set the y axis to be logaritmic
ax.set_yscale('log')
# draw green dot at x = 22 and y = losses[22], losses_z[22] with radius 5
ax.plot(22, losses[22], 'go')
ax.plot(22, losses_z[22], 'go')
# draw green lines at y = losses[22], losses_z[22]
ax.axhline(y=losses[22], color='g', linestyle='-')
ax.axhline(y=losses_z[22], color='g', linestyle='-')
# show the plot
plt.show()

# Sphere - synth - um

### EXPERIMENTAL PARAMETERS

| variable | value |
|----------|:-----:|
| center | (5, 5, 5) um|
| radius | 0.5 um |
| medium | index 1 |
| wavelength | 0.660 um |
| detector grid | 100x100 pixels |
| spacing| 0.1 um |
| c radius | 5 |
| c center-x | 48 |
| c center-y | 48 |

In [2]:
CENTER = (5, 5, 5)
RADIUS = 0.5
MEDIUM_INDEX = 1
WAVELENGTH = 0.660
DETECTOR_SHAPE = [100, 100]
SPACING = 0.1
RECONSTRUCTION_PLANE = 5

sphere = Sphere(
    n=1.59, # refraction of the sphere
    r=RADIUS, 
    center=CENTER 
)

medium_index = MEDIUM_INDEX
illum_wavelen = WAVELENGTH
illum_polarization = (1, 0) # linear polarization along x axis
detector = hp.detector_grid(shape=DETECTOR_SHAPE, 
                            spacing=SPACING) 

holo = calc_holo(
    detector, 
    sphere, 
    medium_index, 
    illum_wavelen, 
    illum_polarization, 
    theory='auto' # use the default theory of sphere
)
hp.show(holo)

zstack = np.linspace(0, 10, 41)
rec_vol = hp.propagate(holo, zstack)
hp.show(rec_vol)

# recontruct the image
recontructed = hp.propagate(holo, RECONSTRUCTION_PLANE - RADIUS)
recontructed = np.abs(recontructed)
recontructed = recontructed.to_numpy()

# draw a circle of radius 5 in the middle of the image
SIZE = recontructed.shape
CIRCLE_RADIUS = 5
CIRCLE_CENTER_X = 48
CIRCLE_CENTER_Y = 48
original = np.zeros(SIZE)
original = draw_circle(CIRCLE_CENTER_X, CIRCLE_CENTER_Y, CIRCLE_RADIUS, original)

# normalize the two images to be between 0 and 1
original = original / np.max(original)
recontructed = recontructed / np.max(recontructed)
hp.show(recontructed)
hp.show(original)

loss, pixel_number = l2_loss(original, recontructed)
print(loss, pixel_number, loss/pixel_number)
loss, pixel_number = l2_loss_zero(original, recontructed)
print(loss, pixel_number, loss/pixel_number)

/Users/emanuelevivoli/miniconda3/envs/mypython3/lib/python3.8/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")


[62.17689512] 10000 [0.00621769]
[61.97878024] 9931 [0.00624094]


# Sphere - synth - um

In [3]:
CENTER = (10, 10, 5)
RADIUS = 0.5
MEDIUM_INDEX = 1
WAVELENGTH = 0.660
DETECTOR_SHAPE = [200, 200]
SPACING = 0.1
RECONSTRUCTION_PLANE = 5

CIRCLE_RADIUS = 5
CIRCLE_CENTER_X = 99
CIRCLE_CENTER_Y = 99

sphere = Sphere(
    n=1.59, # refraction of the sphere
    r=RADIUS, 
    center=CENTER 
)

medium_index = MEDIUM_INDEX
illum_wavelen = WAVELENGTH
illum_polarization = (1, 0) # linear polarization along x axis
detector = hp.detector_grid(shape=DETECTOR_SHAPE, 
                            spacing=SPACING) 

holo = calc_holo(
    detector, 
    sphere, 
    medium_index, 
    illum_wavelen, 
    illum_polarization, 
    theory='auto' # use the default theory of sphere
)
hp.show(holo)

zstack = np.linspace(0, 10, 41)
rec_vol = hp.propagate(holo, zstack)
hp.show(rec_vol)

# recontruct the image
recontructed = hp.propagate(holo, RECONSTRUCTION_PLANE - RADIUS)
recontructed = np.abs(recontructed)
recontructed = recontructed.to_numpy()

# draw a circle of radius 5 in the middle of the image
SIZE = recontructed.shape
original = np.zeros(SIZE)
original = draw_circle(CIRCLE_CENTER_X, CIRCLE_CENTER_Y, CIRCLE_RADIUS, original)
red_recontructed = draw_circle(CIRCLE_CENTER_X, CIRCLE_CENTER_Y, CIRCLE_RADIUS, recontructed.copy())

# normalize the two images to be between 0 and 1
original = original / np.max(original)
recontructed = recontructed / np.max(recontructed)

hp.show(red_recontructed)
hp.show(recontructed)
hp.show(original)

loss, pixel_number = l2_loss(original, recontructed)
print(loss, pixel_number, loss/pixel_number)
loss, pixel_number = l2_loss_zero(original, recontructed)
print(loss, pixel_number, loss/pixel_number)

/Users/emanuelevivoli/miniconda3/envs/mypython3/lib/python3.8/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")


[90.63730305] 40000 [0.00226593]
[90.44307921] 39931 [0.00226498]


# REAL - Sphere - synth - um

In [1]:
CENTER = (13, 15.5, 15)
RADIUS = 10
MEDIUM_INDEX = 1
WAVELENGTH = 15
DETECTOR_SHAPE = [52, 62]
SPACING = 0.5
RECONSTRUCTION_PLANE = CENTER[2]/SPACING

CIRCLE_RADIUS = 10
CIRCLE_CENTER_X = 26
CIRCLE_CENTER_Y = 31

params = {
    'CENTER': CENTER,
    'RADIUS': RADIUS,
    'MEDIUM_INDEX': MEDIUM_INDEX,
    'WAVELENGTH': WAVELENGTH,
    'DETECTOR_SHAPE': DETECTOR_SHAPE,
    'SPACING': SPACING,
    'RECONSTRUCTION_PLANE': RECONSTRUCTION_PLANE,
    'CIRCLE_RADIUS': CIRCLE_RADIUS,
    'CIRCLE_CENTER_X': CIRCLE_CENTER_X,
    'CIRCLE_CENTER_Y': CIRCLE_CENTER_Y
}

def calculate_loss(CENTER, RADIUS, MEDIUM_INDEX, WAVELENGTH, DETECTOR_SHAPE, SPACING, RECONSTRUCTION_PLANE, CIRCLE_RADIUS, CIRCLE_CENTER_X, CIRCLE_CENTER_Y):

    sphere = Sphere(
        n=1.59, # refraction of the sphere
        r=RADIUS, 
        center=CENTER 
    )

    medium_index = MEDIUM_INDEX
    illum_wavelen = WAVELENGTH
    illum_polarization = (1, 0) # linear polarization along x axis
    detector = hp.detector_grid(shape=DETECTOR_SHAPE, 
                                spacing=SPACING) 

    holo = calc_holo(
        detector, 
        sphere, 
        medium_index, 
        illum_wavelen, 
        illum_polarization, 
        theory='auto' # use the default theory of sphere
    )
    # hp.show(holo)

    zstack = np.linspace(0, 60, 121)
    rec_vol = hp.propagate(holo, zstack)
    # hp.show(rec_vol)

    # recontruct the image
    recontructed = hp.propagate(holo, RECONSTRUCTION_PLANE)
    recontructed = np.abs(recontructed)
    recontructed = recontructed.to_numpy()

    # draw a circle of radius 5 in the middle of the image
    SIZE = recontructed.shape
    original = np.zeros(SIZE)
    original = draw_circle(CIRCLE_CENTER_X, CIRCLE_CENTER_Y, CIRCLE_RADIUS, original)

    # normalize the two images to be between 0 and 1
    original = original / np.max(original)
    recontructed = recontructed / np.max(recontructed)
    # hp.show(recontructed)
    # hp.show(original)

    loss, pixel_number = l2_loss(original, recontructed)
    loss_z, pixel_number_z = l2_loss_zero(original, recontructed)

    return loss/pixel_number, loss_z/pixel_number_z

# save losses results
losses = []
losses_z = []
# instantiate the SIZE
SIZE = [52, 62]
# cicle over possible values for SIZE
# use tqdm to show the progress bar
from tqdm import tqdm
for i in tqdm(range(1, 100)):
    SIZE[0] = SIZE[0] + 1
    SIZE[1] = SIZE[1] + 1
    params['DETECTOR_SHAPE'] = SIZE
    results = calculate_loss(**params)
    losses.append(results[0])
    losses_z.append(results[1])

# inlude the matplotlib library
import matplotlib.pyplot as plt

# draw the results in line plot with two colors
fig, ax = plt.subplots()
ax.plot(losses, color='red')
ax.plot(losses_z, color='blue')
ax.set_xlabel('SIZE')
ax.set_ylabel('loss')
ax.set_title('losses')
plt.show()


  0%|          | 0/99 [00:00<?, ?it/s]


NameError: name 'Sphere' is not defined

# REAL - Sphere - synth - um

In [5]:
CENTER = (26, 31, 15)
RADIUS = 10
MEDIUM_INDEX = 1
WAVELENGTH = 15
DETECTOR_SHAPE = [104, 124]
SPACING = 0.5
RECONSTRUCTION_PLANE = CENTER[2]/SPACING

CIRCLE_RADIUS = 10
CIRCLE_CENTER_X = 52
CIRCLE_CENTER_Y = 62

sphere = Sphere(
    n=1.59, # refraction of the sphere
    r=RADIUS, 
    center=CENTER 
)

medium_index = MEDIUM_INDEX
illum_wavelen = WAVELENGTH
illum_polarization = (1, 0) # linear polarization along x axis
detector = hp.detector_grid(shape=DETECTOR_SHAPE, 
                            spacing=SPACING) 

holo = calc_holo(
    detector, 
    sphere, 
    medium_index, 
    illum_wavelen, 
    illum_polarization, 
    theory='auto' # use the default theory of sphere
)
hp.show(holo)

zstack = np.linspace(0, 60, 121)
rec_vol = hp.propagate(holo, zstack)
hp.show(rec_vol)

# recontruct the image
recontructed = hp.propagate(holo, RECONSTRUCTION_PLANE)
recontructed = np.abs(recontructed)
recontructed = recontructed.to_numpy()

# draw a circle of radius 5 in the middle of the image
SIZE = recontructed.shape
original = np.zeros(SIZE)
original = draw_circle(CIRCLE_CENTER_X, CIRCLE_CENTER_Y, CIRCLE_RADIUS, original)

# normalize the two images to be between 0 and 1
original = original / np.max(original)
recontructed = recontructed / np.max(recontructed)
hp.show(recontructed)
hp.show(original)

loss, pixel_number = l2_loss(original, recontructed)
print(loss, pixel_number, loss/pixel_number)
loss, pixel_number = l2_loss_zero(original, recontructed)
print(loss, pixel_number, loss/pixel_number)

/Users/emanuelevivoli/miniconda3/envs/mypython3/lib/python3.8/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")


[38.55759873] 12896 [0.00298989]
[37.00446755] 12591 [0.00293896]


# REAL - VESSEL - cm

## utils vessel

In [4]:
from PIL import Image, ImageDraw

def draw_vessel(img, center_x=24, center_y=26, size_x=19, size_y=9.5, radius=4, fill=(255, 255, 255)):

    # create a drawing context
    draw = ImageDraw.Draw(img)

    # calculate the coordinates of the corners of the rectangle
    # relative to the center of the image
    y1 = -size_y / 2
    x1 = -size_x / 2
    y2 = size_y / 2
    x2 = size_x / 2

    # draw the rectangle with rounded corners
    draw.rounded_rectangle([(center_x + x1, center_y + y1), (center_x + x2, center_y + y2)],
                fill=fill,
                radius=radius)

    # get the resulting image as numpy array
    original_rect = np.array(img)
    
    return original_rect

## loss vessel

In [5]:
CENTER = 7.5
MEDIUM_INDEX = 1
WAVELENGTH = 15
SPACING = 0.5
RECONSTRUCTION_PLANE = CENTER/SPACING
DETECTOR_SHAPE = [52, 62]

CIRCLE_RADIUS = 10
CIRCLE_CENTER_X = 52
CIRCLE_CENTER_Y = 62

custom_holo = np.load("/Users/emanuelevivoli/asmara/data/interim/holograms/indoor/in_bas_13_D_6.npy")
hp.show(custom_holo)

# save image to file
I8 = (((custom_holo - custom_holo.min()) / (custom_holo.max() - custom_holo.min())) * 255).astype(np.uint8)
img = Image.fromarray(I8)
img.save("file.png")

# load image from file
raw_holo = hp.load_image("file.png", 
        medium_index=MEDIUM_INDEX, 
        illum_wavelen=WAVELENGTH,
        illum_polarization=(1,0), 
        spacing=SPACING,
        channel=[0,1,2])


zstack = np.linspace(0, 30, 61)
rec_vol = hp.propagate(raw_holo, zstack)
hp.show(rec_vol)


# recontruct the image
recontructed = hp.propagate(raw_holo, RECONSTRUCTION_PLANE)
recontructed = np.abs(recontructed)
recontructed = recontructed.to_numpy()
hp.show(recontructed)

# draw a circle of radius 5 in the middle of the image
SIZE = recontructed.shape
# create an image with a black background
original = Image.new('RGB', (62, 52), color=(0, 0, 0))
original = draw_vessel(original)

# print shapes of the images
print(original.shape, recontructed.shape)
# transform RGB color image original to grayscale
original = np.dot(original[...,:3], [0.299, 0.587, 0.114])
print(original.shape, recontructed.shape)
# squeeze recontructed the image to 2D
recontructed = np.squeeze(recontructed, axis=2)
print(original.shape, recontructed.shape)

red_recontructed = Image.fromarray(recontructed)
red_recontructed = draw_vessel(red_recontructed, fill=None)

# normalize the two images to be between 0 and 1
original = original / np.max(original)
recontructed = recontructed / np.max(recontructed)
red_recontructed = red_recontructed / np.max(red_recontructed)
hp.show(recontructed)
hp.show(red_recontructed)
hp.show(original)

loss, pixel_number = l2_loss(original, recontructed)
print(loss, pixel_number, loss/pixel_number)
loss, pixel_number = l2_loss_zero(original, recontructed)
print(loss, pixel_number, loss/pixel_number)

/Users/emanuelevivoli/miniconda3/envs/mypython3/lib/python3.8/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")
/var/folders/yw/39qgg96x2451prd0f9qrtbzr0000gn/T/ipykernel_71481/3789391024.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  I8 = (((custom_holo - custom_holo.min()) / (custom_holo.max() - custom_holo.min())) * 255).astype(np.uint8)
/Users/emanuelevivoli/miniconda3/envs/mypython3/lib/python3.8/site-packages/holopy/core/io/io.py:251: UserWarning: Not a color image (channel number ignored)
  warnings.warn("Not a color image (channel number ignored)")
/Users/emanuelevivoli/miniconda3/envs/mypython3/lib/python3.8/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")


(52, 62, 3) (52, 62, 1)
(52, 62) (52, 62, 1)
(52, 62) (52, 62)
32.63485467433612 3224 0.010122473534223362
32.459804651180306 3044 0.010663536350584858


pixels: x = 41, y = 24, z = 0, units: x = 4.1e+01, y = 2.4e+01, z = 0.0e+00,
pixels: x = 20, y = 33, z = 16, units: x = 9.8e+00, y = 1.7e+01, z = 8.0e+00,
pixels: x = 35, y = 33, z = 0, units: x = 3.5e+01, y = 3.3e+01, z = 0.0e+00,


2022-12-16 16:19:22.135 python[71481:1774069] +[CATransaction synchronize] called within transaction
2022-12-16 16:19:23.545 python[71481:1774069] +[CATransaction synchronize] called within transaction


pixels: x = 32, y = 54, z = 0, units: x = 3.2e+01, y = 5.4e+01, z = 0.0e+00,


# REAL PMN4 in the image

In [10]:
from PIL import Image, ImageDraw

def draw_pmn4(img, center_x=19, center_y=24, size_x=9.5, size_y=9.5, fill=(255, 255, 255)):

    # create a drawing context
    draw = ImageDraw.Draw(img)

    # calculate the coordinates of the corners of the rectangle
    # relative to the center of the image
    y1 = -size_y / 2
    x1 = -size_x / 2
    y2 = size_y / 2
    x2 = size_x / 2

    # draw the rectangle with rounded corners
    draw.ellipse([(center_x + x1, center_y + y1), (center_x + x2, center_y + y2)],
                fill=fill)

    # get the resulting image as numpy array
    original_circ = np.array(img)
    
    return original_circ

In [11]:
CENTER = 7.5
MEDIUM_INDEX = 1
WAVELENGTH = 15
SPACING = 0.5
RECONSTRUCTION_PLANE = CENTER/SPACING
DETECTOR_SHAPE = [52, 62]

CIRCLE_RADIUS = 10
CIRCLE_CENTER_X = 52
CIRCLE_CENTER_Y = 62

custom_holo = np.load("/Users/emanuelevivoli/asmara/data/interim/holograms/indoor/in_bas_01_A_6.npy")
hp.show(custom_holo)

# save image to file
I8 = (((custom_holo - custom_holo.min()) / (custom_holo.max() - custom_holo.min())) * 255).astype(np.uint8)
img = Image.fromarray(I8)
img.save("file.png")

# load image from file
raw_holo = hp.load_image("file.png", 
        medium_index=MEDIUM_INDEX, 
        illum_wavelen=WAVELENGTH,
        illum_polarization=(1,0), 
        spacing=SPACING,
        channel=[0,1,2])


zstack = np.linspace(0, 30, 61)
rec_vol = hp.propagate(raw_holo, zstack)
hp.show(rec_vol)


# recontruct the image
recontructed = hp.propagate(raw_holo, RECONSTRUCTION_PLANE)
recontructed = np.abs(recontructed)
recontructed = recontructed.to_numpy()
hp.show(recontructed)

# draw a circle of radius 5 in the middle of the image
SIZE = recontructed.shape
# create an image with a black background
original = Image.new('RGB', (62, 52), color=(0, 0, 0))
original = draw_pmn4(original)

# print shapes of the images
print(original.shape, recontructed.shape)
# transform RGB color image original to grayscale
original = np.dot(original[...,:3], [0.299, 0.587, 0.114])
print(original.shape, recontructed.shape)
# squeeze recontructed the image to 2D
recontructed = np.squeeze(recontructed, axis=2)
print(original.shape, recontructed.shape)

red_recontructed = Image.fromarray(recontructed)
red_recontructed = draw_pmn4(red_recontructed, fill=None)

# normalize the two images to be between 0 and 1
original = original / np.max(original)
recontructed = recontructed / np.max(recontructed)
red_recontructed = red_recontructed / np.max(red_recontructed)
hp.show(recontructed)
hp.show(red_recontructed)
hp.show(original)

loss, pixel_number = l2_loss(original, recontructed)
print(loss, pixel_number, loss/pixel_number)
loss, pixel_number = l2_loss_zero(original, recontructed)
print(loss, pixel_number, loss/pixel_number)

/Users/emanuelevivoli/miniconda3/envs/mypython3/lib/python3.8/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")
/var/folders/yw/39qgg96x2451prd0f9qrtbzr0000gn/T/ipykernel_45019/2185214691.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  I8 = (((custom_holo - custom_holo.min()) / (custom_holo.max() - custom_holo.min())) * 255).astype(np.uint8)
/Users/emanuelevivoli/miniconda3/envs/mypython3/lib/python3.8/site-packages/holopy/core/io/io.py:251: UserWarning: Not a color image (channel number ignored)
  warnings.warn("Not a color image (channel number ignored)")
/Users/emanuelevivoli/miniconda3/envs/mypython3/lib/python3.8/site-packages/holopy/core/io/vis.py:264: UserWarning: Image contains complex values. Taking image magnitude.
  warn("Image contains complex values. Taking image magnitude.")


(52, 62, 3) (52, 62, 1)
(52, 62) (52, 62, 1)
(52, 62) (52, 62)
38.12358613489284 3224 0.011824933664668994
38.02563565358553 3148 0.012079299762892482


pixels: x = 4, y = 49, z = 0, units: x = 3.7e+00, y = 4.9e+01, z = 0.0e+00,
